# Let's take a look at some hoot-larious games in the NFL.  
### We'll be looking at:  
- Games where no touchdowns were scored  
- Games where the winning team did not score a touchdown  
- Games where both teams scores were single digits 
***  
If anything, this will be useful to make sure we don't watch any boring games.
***

In [1]:
import pandas as pd
import numpy as np

In [2]:
#get a list of years
YEARS=list(range(1999,2021))
#get the list of data fields used
fields= ['game_id', 'touchdown', 'td_team', 'home_score','away_score', 'home_team','away_team','season']
displayfields = ['game_id','season', 'gamewinner', 'loser', 'score']

#create empty dataframes for each dataset
noTD = pd.DataFrame()
noTDwinner = pd.DataFrame()
singledigitgame = pd.DataFrame()

for i in YEARS:
    #read each file into a data set
    dataset = pd.read_csv(f'{i} nfl data.csv', usecols=fields)

    #add a column 'gamewinner' and populate
    #game winner, losers, tie
    dataset.loc[dataset['home_score'] > dataset['away_score'], 'gamewinner'] = dataset['home_team']
    dataset.loc[dataset['home_score'] < dataset['away_score'], 'gamewinner'] = dataset['away_team']
    dataset.loc[dataset['home_score'] == dataset['away_score'], 'gamewinner'] = 'tie'
    
    dataset.loc[dataset['home_score'] > dataset['away_score'], 'loser'] = dataset['away_team']
    dataset.loc[dataset['home_score'] < dataset['away_score'], 'loser'] = dataset['home_team']
    dataset.loc[dataset['home_score'] == dataset['away_score'], 'loser'] = 'tie'
    
    #create a score field
    dataset['score'] = np.where((dataset['home_score'] > dataset['away_score']) | \
        (dataset['home_score'] == dataset['away_score']), \
        (dataset["home_score"].astype(str) + " - " + dataset["away_score"].astype(str)), \
        (dataset["away_score"].astype(str) + " - " + dataset["home_score"].astype(str)))

    
    #df1 is the dataframe for all the games with zero touchdowns
    df1=pd.DataFrame(dataset.groupby(displayfields)['touchdown'].sum())
    df1=df1.loc[df1['touchdown']==0]
    df1.reset_index(inplace=True)
    
    #df2 is the dataframe for all games where the winning team did not score a TD
    df2= pd.DataFrame(dataset.groupby(displayfields)['td_team'].unique())
    df2.reset_index(inplace=True)

    #create a field ('flag'). If the game winner is in the 'td_team' field, populate 1. Otherwise, 0
    df2['flag'] = df2.apply(lambda x: int(x['gamewinner'] in x['td_team']), axis=1)

    #filter for all the zeros
    df2=df2.loc[df2['flag']==0]

    #df3 is for all games where the winning team scored since digits
    df3 = pd.pivot_table(dataset, index=['game_id', 'gamewinner','loser','score'])
    df3=df3.loc[(df3['away_score']<10) & (df3['home_score']<10) ]
    df3.reset_index(inplace=True)
    
    #concat datasets
    noTD = pd.concat([noTD, df1])
    noTDwinner = pd.concat([noTDwinner, df2])
    singledigitgame = pd.concat([singledigitgame, df3])
    
#delete all ties from noTDwinner
noTDwinner.drop(noTDwinner[noTDwinner['gamewinner'] == 'tie'].index, inplace = True)

#delete noTD games from noTDwinner
#noTDwinner = noTDwinner[~noTDwinner['game_id'].isin(noTD['game_id'])]

dfs = [noTD, noTDwinner, singledigitgame]
for i in dfs:
    i.reset_index(inplace=True, drop=True)

C:\Users\Anthony Celia\AppData\Local\Temp\ipykernel_8516\1133034709.py:14: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(f'{i} nfl data.csv', usecols=fields)


## A List of Games Without a Touchdown

In [3]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: white !important;
}
</style>

In [4]:
noTD[displayfields]

,game_id,season,gamewinner,loser,score
0,1999_07_CHI_TB,1999,TB,CHI,6 - 3
1,1999_10_BAL_JAX,1999,JAX,BAL,6 - 3
2,1999_15_SD_MIA,1999,MIA,LAC,12 - 9
3,2000_03_PHI_GB,2000,GB,PHI,6 - 3
4,2000_05_BAL_CLE,2000,BAL,CLE,12 - 0
5,2001_01_SEA_CLE,2001,SEA,CLE,9 - 6
6,2001_14_NE_BUF,2001,NE,BUF,12 - 9
7,2002_03_SEA_NYG,2002,NYG,SEA,9 - 6
8,2002_08_TB_CAR,2002,TB,CAR,12 - 9
9,2002_17_TB_CHI,2002,TB,CHI,15 - 0


## A List of Games Where the Winner Did Not Score a Touchdown

In [5]:
noTDwinner[displayfields]

,game_id,season,gamewinner,loser,score
0,1999_07_CHI_TB,1999,TB,CHI,6 - 3
1,1999_10_BAL_JAX,1999,JAX,BAL,6 - 3
2,1999_15_SD_MIA,1999,MIA,LAC,12 - 9
3,2000_02_WAS_DET,2000,DET,WAS,15 - 10
4,2000_03_PHI_GB,2000,GB,PHI,6 - 3
...,...,...,...,...,...
63,2018_13_IND_JAX,2018,JAX,IND,6 - 0
64,2019_04_DAL_NO,2019,NO,DAL,12 - 10
65,2019_07_SF_WAS,2019,SF,WAS,9 - 0
66,2020_06_DEN_NE,2020,DEN,NE,18 - 12


## A List of Games Where Neither Team Scored Double Digits

In [6]:
singledigitgame[displayfields]

,game_id,season,gamewinner,loser,score
0,1999_07_CHI_TB,1999,TB,CHI,6 - 3
1,1999_10_BAL_JAX,1999,JAX,BAL,6 - 3
2,2000_01_SD_OAK,2000,LV,LAC,9 - 6
3,2000_03_PHI_GB,2000,GB,PHI,6 - 3
4,2000_09_PIT_BAL,2000,PIT,BAL,9 - 6
5,2000_10_PIT_TEN,2000,TEN,PIT,9 - 7
6,2000_14_NYG_WAS,2000,NYG,WAS,9 - 7
7,2001_01_SEA_CLE,2001,SEA,CLE,9 - 6
8,2001_05_WAS_DAL,2001,DAL,WAS,9 - 7
9,2002_03_SEA_NYG,2002,NYG,SEA,9 - 6
